In [2]:
# %load dataloaderraw.py
from __future__ import division

import os
import torch
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset

class ImageSpecDataset(Dataset):
    """Dataset consisting of image files ans spectrogram images of audio files."""

    def __init__(self, root_dir, split = 'train'):
        """
        Args:
            root_dir (string): Directory with all the images.
        """
        csv_file = os.path.join(root_dir, split + '_split.tsv')
        self.file_list = []
        for line in open(csv_file, 'r').readlines()[1:]:
            self.file_list.append(line.split('\t'))
        self.root_dir = root_dir
        self.split = split

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.split, 'img_feats', self.file_list[idx][0]+'.npy')
        image_feats = np.load(img_name)
        specs_name = os.path.join(self.root_dir, self.split, 'spec', self.file_list[idx][0]+'.png')
        spectrogram = io.imread(specs_name, as_gray=True) / 255.0
        spectrogram = np.asarray(spectrogram)
        score = self.file_list[idx][11]
        sample = {'img_feats': image_feats, 'spec': spectrogram, 'score': float(score)}

        return sample
